In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


https://medium.com/analytics-vidhya/pytorch-for-deep-learning-binary-classification-logistic-regression-382abd97fb43

In [5]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
df = pd.read_csv("gdrive/MyDrive/DataScienceProjectData/ERisk_data.csv")
df = df.dropna()
y = df['label']
x = df.iloc[:,1:]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [8]:
#defining dataset class
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(np.array(x),dtype=torch.float32)
    self.y = torch.tensor(np.array(y),dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

trainset = dataset(x_train,y_train)
testset = dataset(x_test,y_test)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [9]:
#defining the network
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,512)
    self.fc2 = nn.Linear(512,1028)
    self.fc3 = nn.Linear(1028,256)
    self.fc4 = nn.Linear(256,1)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.sigmoid(self.fc4(x))
    return x

In [10]:
#hyper parameters
learning_rate = 0.01
epochs = 9000
# Model , Optimizer, Loss
model = Net(input_shape=x.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
loss_fn = nn.BCELoss()

In [20]:
#forward loop
losses = []
accur = []
for i in range(1):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = model(x_train.to(device))
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1).to(device))
 
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  if i%50 == 0:
    #prediction
    predicted = model(torch.tensor(np.array(x_test),dtype=torch.float32).to(device))
    # wrap y_test in numpy array
    #loss
    test_loss = loss_fn(predicted,y_test.reshape(-1,1).to(device) )

    # convert to numpy
    predicted = predicted.reshape(-1).detach().cpu().numpy()

    # 0.5 cut off here
    fpr, tpr, thresholds = metrics.roc_curve(np.array(y_test), predicted)
    auc = metrics.auc(fpr, tpr)
    accur.append(auc)
    print("epoch {}\t training loss : {}\t test auc : {}".format(i,loss,auc))

AttributeError: ignored

In [12]:
if 2

416     0.0
1326    0.0
1351    1.0
1640    1.0
1257    0.0
       ... 
764     1.0
79      1.0
571     1.0
80      1.0
1398    1.0
Name: label, Length: 293, dtype: float64